## Goal for next update:
1. Adapt the base Ramsey_parallel code so that it searches multiple colorings. 

## Idea behind this update:
Compared to the amount of space taken up by the subgraph index array, a coloring takes up minimal space. Also, for R(4,6) and R(5,5), we have plenty of space left on the GPU's global array after sending down the complete subgraph index array. So, we should be able to send down a bunch of colorings in order to perform many independent Markov searches without having to adapt our code significantly. 

## NVIDIA DOC:
http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#maximize-instruction-throughput

LIMITS FOR THE DATA TYPES:

#int8 	Byte (-128 to 127)
#int16 	Integer (-32768 to 32767)
#int32 	Integer (-2147483648 to 2147483647)
#int64 	Integer (-9223372036854775808 to 9223372036854775807)
#uint8 	Unsigned integer (0 to 255)
#uint16 	Unsigned integer (0 to 65535)
#uint32 	Unsigned integer (0 to 4294967295)
#uint64 	Unsigned integer (0 to 18446744073709551615)

CONVERSION USING NUMPY:

(unsigned) char = numpy.(u)int8

(unsigned) short = numpy.(u)int16

(unsigned) int = numpy.(u)int32

(unsigned) long = numpy.(u)int64 (only 64-bit)

floats = numpy.float32

double = numpy.float64

all pointers ( e.g int *, float ***, anything at all) should be numpy.intp.

In [ ]:
from setup import *
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import random

def main_random(Ramsey, num_vertices, num_steps, num_colorings = 5, beta=2):
    start_time = datetime.datetime.now()
    print()

    def choose(n,k):
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
    #The theory for this function comes from the Lesser Paper. Theorem 7 there provides very useful lower-bounds 
    #for the number of red edges found 
    def min_red_edges(Ramsey, num_vertices):
        sorted_Ramsey = np.sort(Ramsey)
        k = sorted_Ramsey[1]
        if len(sorted_Ramsey) != 2 or sorted_Ramsey[0] != 3:
            return(0)
        elif num_vertices <=2*k:
            return(num_vertices-k)
        elif num_vertices <= 5*k/2:
            return(3*num_vertices - 5*k)
        else:
            return(5*num_vertices - 10*k)
    min_red = min_red_edges(Ramsey, num_vertices)
    num_colors = len(Ramsey)
    Colors = np.arange(num_colors)
    Vertices = np.arange(num_vertices)
    Edges = list(it.combinations(Vertices,2))
    #reverse lookup for edges below.  Eg if slot 3 above contains edge (2,5), the the dict below has entry (2,5):3
    Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges)) 
    num_edges = len(Edges)

    threads_per_block = 1024
    vertices_per_clique = Ramsey
    edges_per_clique = np.array([choose(v,2) for v in vertices_per_clique])
    cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
    blocks_per_color = np.ceil(cliques_per_color / threads_per_block).astype('uint32')
    num_blocks = blocks_per_color.sum()
    cliques_per_block = np.ceil(cliques_per_color / blocks_per_color).astype('uint32')
    #The objects below tells each block which color and how many cliques/edges it will monitor.
    #Note each vector is repetitive.  If color 0 gets 7 blocks, the first 7 entries will be the same
    block_color = np.repeat(Colors,blocks_per_color).astype('uint32')
    block_num_cliques = np.repeat(cliques_per_block,blocks_per_color).astype('uint32')
    block_edges_per_clique = np.repeat(edges_per_clique,blocks_per_color).astype('uint32')
    num_threads_per_coloring = num_blocks*threads_per_block
    
    assign_Blocks_to_Cliques = np.full([num_blocks,cliques_per_block.max(),edges_per_clique.max()],
                                       fill_value=num_edges, dtype='uint32')

    #Counters that that tracks the next open block and thread on each block
    next_open_block = 0    
    next_open_thread = np.zeros(num_blocks,dtype='int')
    for color, clique_size in enumerate(Ramsey):
        #Creates a generator to produce all cliques (the list of vertices).
        Cliques = it.combinations(Vertices,clique_size)
        #Makes the vector [0,1,2,...,num_blocks-1,0,1,2,...,num_blocks-1,....] of length num_cliques
        assign_Cliques_to_Blocks = np.arange(cliques_per_color[color]) % blocks_per_color[color]
        #randomizes assignment, but maintains clique counts
        np.random.shuffle(assign_Cliques_to_Blocks)
        #Starts at next open block
        assign_Cliques_to_Blocks += next_open_block
        
        for clique_Vertices, block in zip(Cliques,assign_Cliques_to_Blocks):
            #Gets the list of edges in this clique
            clique_Edges = list(it.combinations(clique_Vertices,2))
            #Converts it to edge_idx
            clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]
            #Writes it to the correct block and next open thread on that block
            assign_Blocks_to_Cliques[block,next_open_thread[block],:edges_per_clique[color]] = clique_Edges_idx            
            next_open_thread[block] += 1
        next_open_block += blocks_per_color[color]

    #Code below sets up the GPU checker
    block_color_gpu = gpuarray.to_gpu(block_color)
    #block_num_cliques = gpuarray.to_gpu(block_num_cliques)
    block_edges_per_clique_gpu = gpuarray.to_gpu(block_edges_per_clique)
    assign_Blocks_to_Cliques_gpu = gpuarray.to_gpu(assign_Blocks_to_Cliques)    

    kernel_code ="""
    #include <stdio.h>
    __global__ void find_problems(int *block_color, int *edges_per_clique, int *edges, int *coloring, int *Problems, int edges_per_thread)//, int cliques)
    {
        __shared__ int shared_coloring[shared_size];
        //This marches first down the x-dimension, then the y-dimension and finally the z-dimension. I'm not positive that
        //this is the best way to allocate the blocks but it is simple and easy to implement. 
        //Also, for the sake of simplicity, I've made the blocks only 1-D.
        int Id = threadIdx.x + blockDim.x*blockIdx.x + blockDim.x*gridDim.x*blockIdx.y + blockDim.x*gridDim.x*blockDim.y*gridDim.y*blockIdx.z;
        
        //If Id < num_threads_per_coloring, it is a thread that deals with the first coloring. 
        //If num_threads_per_coloring =< Id < 2*num_threads_per_coloring, then it deals with the second coloring and etc.
        int coloring_Id = (int) Id/num_threads_per_coloring;
        
        //Also walks down x, y and z dimensions respectively.
        int blockid = blockIdx.x + gridDim.x*blockIdx.y + gridDim.x*gridDim.y*blockIdx.z;
        
        //These next two numbers are how the threads navigate the assignment array.
        int BlockId = blockid - num_blocks_per_coloring*coloring_Id;
        int CliqueId = Id - num_threads_per_coloring*coloring_Id;
        //color that the thread is looking for
        int color = block_color[BlockId];
        if(threadIdx.x < shared_size)
        {
            shared_coloring[threadIdx.x] = coloring[coloring_Id*(shared_size-1) + threadIdx.x];
        }
        //if(clique_idx < cliques)
        //{
            //This weird way of starting and stopping allows us to traverse the 3D-array of assign_Blocks_to_Cliques easily
            int start = CliqueId*edges_per_thread;
            int end = start + edges_per_clique[BlockId];
            int e = start;
            while((e < end) && (shared_coloring[edges[e]] == color))
            {
                e++;
            }
            //Problems[clique_idx] = (e >= end) ? 1 : 0;
            if(e>=end)
            {
                atomicAdd(&Problems[coloring_Id], 1);
            }
        //}
    }
    """
    
    regex_fixing_dictionary = {"shared_size":(num_edges+1), "num_threads_per_coloring":(num_threads_per_coloring), 
                              "num_blocks_per_coloring":num_blocks}

    for key, val in regex_fixing_dictionary.items():
        kernel_code = kernel_code.replace(str(key),str(val))
    mod = SourceModule(kernel_code)
    G, B, edges_per_thread = assign_Blocks_to_Cliques_gpu.shape
    num_blocks_total = G*num_colorings
    #Creating a grid that is a cube. The easiest way to do it is to find smallest possible cube that fits it and just
    #allocate two many blocks. Later, I might experiment with other shapes to see if it affects it some.
    grid_dimensions = (int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1,int(num_blocks_total**(1/3)) + 1)
    block_dimensions = (B,1,1)
    edges_per_thread = np.uint32(edges_per_thread)
    print("#blocks = gridDim.x = %d, cliques per block = threads per block = blockDim.x = %d, edges per thread = %d"%(G,B,edges_per_thread))
    func = mod.get_function("find_problems")
    #Note, the current vesion has cuda add up the number of problems on a coloring all by itself and returns an array
    #where the first element is the number of problems in the first coloring, the second element is the number of problems
    #in the second coloring and etc.
    def find_problems_cuda(coloring_gpu, printout=False):
        Problems_gpu = gpuarray.to_gpu(np.zeros(num_colorings).astype("int32"))
        func(block_color_gpu, block_edges_per_clique_gpu, assign_Blocks_to_Cliques_gpu, coloring_gpu, Problems_gpu, edges_per_thread, block=block_dimensions, grid=grid_dimensions, shared=0)
        Problems_cpu = Problems_gpu.get().astype("int32")
        if printout == True:
            print_problems(Problems_cpu)
        return Problems_cpu
#   Here the start of the Pandas code. Used to double check my find_problems_cuda

    #compare = np.full_like(assign_Blocks_to_Cliques, fill_value=num_colors)
    #print(assign_Blocks_to_Cliques.shape)
    #for block in range(num_blocks):
    #    compare[block,:,:block_edges_per_clique[block]] = block_color[block]
    #
    #def find_problems_pandas(coloring, printout=False):
    #    X = coloring[assign_Blocks_to_Cliques]
    #    Y = (X == compare)
    #    Problems = np.all(Y,axis=-1)
    #    if printout == True:
    #        print_problems(Problems)
    #    return Problems.sum().astype('int'), Problems

    def print_problems(Problems):        
        Z = pd.DataFrame(Problems.astype('uint32'))
        Z.insert(0,'problems',Z.sum(axis=1))
        Z.insert(0,'color',block_color)
        Z = Z.T
        problem_idx = np.any(Z,axis=-1)
        problem_idx[:2] = True
        display(Z.ix[problem_idx,:])

    def print_status():
        now = datetime.datetime.now()
        elapsed = now - start_time
        print("%d steps done in %s.  Best coloring so far was coloring %d in step %d with %d problems.  Time now %s."
                   %(step,str(elapsed).split('.')[0],best_coloring_id, step_best,num_problems_best,str(now).split('.')[0]))
    def increase_red_edges(coloring):
        while(list(coloring).count(red)<min_red):
            idx = [i for i in range(len(coloring)) if coloring[i]!=red]
            coloring[random.choice(idx)] = red
        return(coloring)
    #Initialize the Markov chain
    coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
    np.random.shuffle(coloring_cpu)
    for coloring in range(num_colorings):
        coloring_cpu[coloring][num_edges] = num_colors
    coloring_best = coloring_cpu.copy()
    coloring_gpu = gpuarray.to_gpu(coloring_cpu.copy())

    #num_problems_current, Problems_current = find_problems_pandas(coloring_cpu, printout=False)
    #num_problems_current, Problems_current = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
    num_problems_current = find_problems_cuda(coloring_gpu)    
    num_problems_proposed = num_problems_current    
    num_problems_best = np.min(num_problems_current)
    best_coloring_id = np.argmin(num_problems_current)
    coloring_best = coloring_cpu[best_coloring_id].copy()
    red = min(vertices_per_clique)
    step = 0
    step_best = step
    
    loop_length = 100000
    loop_step = 0
    loops_done = 0
    start_compute = datetime.datetime.now()
    for i in range(num_steps):
        if num_problems_best == 0:
            break
        #This creates a random edge to switch, a random amount to change it by and an array of each of the old edge
        #colors. 
        #coloring_cpu = np.array([np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32') for coloring in range(num_colorings)])
        edge_idx = np.array([np.random.randint(0,num_edges) for coloring in range(num_colorings)])
        color_delta = np.array([np.random.randint(1,num_colors) for coloring in range(num_colorings)])
        edge_color_old = np.ones(num_colorings)*-1
        #For each coloring, find edge to switch, amount to switch by, actually change it and then ensure it has 
        #enough red edges
        for coloring in range(num_colorings):
            current_edge = edge_idx[coloring]
            edge_color_old[coloring] = coloring_cpu[coloring][current_edge]
            coloring_cpu[coloring][current_edge] = (edge_color_old[coloring] + color_delta[coloring])%num_colors
            if(list(coloring_cpu[coloring]).count(red) < min_red):
                coloring_cpu[coloring] = increase_red_edges(coloring_cpu[coloring])    
        coloring_gpu.set(coloring_cpu)

#         The code below check the pandas and cuda versions against each other.
#         It is commented out by default because it slows things down.
#         If you want to use it, you also need to uncomment several lines above to activate the pandas algorithm.

        #num_problems_proposed, Problems_proposed_pandas = find_problems_pandas(coloring_cpu)#, printout=True)
        #num_problems_proposed, Problems_proposed_cuda = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
        #if np.all(Problems_proposed_pandas == Problems_proposed_cuda) == True:
        #    print("Pandas and Cuda agree!!")
        #else:
        #    raise Exception("Pandas and Cuda disagree :()")
            
        num_problems_proposed = find_problems_cuda(coloring_gpu)
        num_problems_diff = num_problems_current - num_problems_proposed
        #print("current:",num_problems_current,"proposed", num_problems_proposed)
        for coloring in range(num_colors):
            current_edge = edge_idx[coloring]
            if num_problems_diff[coloring] >= 0:
                #print("Proposed is better.  Accepting.")            
                num_problems_current[coloring] = num_problems_proposed[coloring]
                #Problems_current = Problems_proposed.copy()
                #if num_problems_proposed[coloring] < num_problems_best:
                #    step_best = step
                #    coloring_best = coloring_cpu[coloring].copy()
                #    num_problems_best = num_problems_proposed
                #    #Problems_best = Problems_proposed.copy()
                #    print_status()
            else:            
                accept = np.exp(beta * num_problems_diff[coloring])            
                r = np.random.random()
                #print("Proposed is worse.") # But I will accept it anyway if I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
                if r <= accept:            
                    #print("So I accept the move even though it is worse.")                
                    num_problems_current[coloring] = num_problems_proposed[coloring]
                    #Problems_current = Problems_proposed.copy()
                else:                
                    #print("So I reject.")
                    coloring_cpu[coloring][current_edge] = edge_color_old[coloring]
        if np.min(num_problems_current)< num_problems_best:
            step_best = step
            best_coloring_id = np.argmin(num_problems_current)
            coloring_best = coloring_cpu[best_coloring_id].copy()
            num_problems_best = num_problems_current[best_coloring_id]
            print_status()
        step += 1
        loop_step += 1
        if(loop_step >= loop_length):
            loops_done += 1
            loop_step = 0
            print_status()
            compute_time = (datetime.datetime.now() - start_compute).seconds
            steps_done = loops_done*loop_length
            rate = steps_done*num_colorings / compute_time
            job_time = (num_steps-steps_done)/rate
            m, s = divmod(job_time,60)
            h, m = divmod(m,60)
            d, h = divmod(h,24)
            y, d = divmod(d,365)
            print("At %.0f colorings/second, it'll take me %d years %d days %d hours %d minutes and %d seconds to complete the remaining steps."%
                  (rate,y,d,h,m,s))

    print("FINISHED!!")
    #coloring_cpu = coloring_best.copy()
    #coloring_gpu.set(coloring_best)
    #num_problems_best = find_problems_cuda(coloring_gpu)
    print()
    print_status()
    final_coloring = pd.DataFrame()
    final_coloring['edge'] = Edges
    final_coloring['color'] = coloring_best[:num_edges]
    display(final_coloring)
    return final_coloring

In [ ]:
bill = np.array([range(i,4+i) for i in range(4)])
np.random.shuffle(bill)
print(bill)

In [3]:
Ramsey = [4,6]
num_vertices = 35
num_colorings = 1*10**3
num_steps = 3*10**4
import datetime 
#Ramsey = [3,3,4]
#num_vertices = 30
#num_steps = 1000000000

beta = 1
bill = main_random(Ramsey, num_vertices, num_steps, num_colorings, beta=beta)


#blocks = gridDim.x = 1638, cliques per block = threads per block = blockDim.x = 1024, edges per thread = 15
44 steps done in 0:01:23.  Best coloring so far was coloring 0 in step 44 with 591 problems.  Time now 2017-07-06 16:07:30.
45 steps done in 0:01:25.  Best coloring so far was coloring 0 in step 45 with 575 problems.  Time now 2017-07-06 16:07:32.
49 steps done in 0:01:31.  Best coloring so far was coloring 0 in step 49 with 574 problems.  Time now 2017-07-06 16:07:38.
52 steps done in 0:01:36.  Best coloring so far was coloring 0 in step 52 with 570 problems.  Time now 2017-07-06 16:07:43.
55 steps done in 0:01:41.  Best coloring so far was coloring 0 in step 55 with 564 problems.  Time now 2017-07-06 16:07:48.
56 steps done in 0:01:43.  Best coloring so far was coloring 0 in step 56 with 558 problems.  Time now 2017-07-06 16:07:50.
59 steps done in 0:01:47.  Best coloring so far was coloring 0 in step 59 with 545 problems.  Time now 2017-07-06 16:07:55.
62 steps done in 0:01:

2560 steps done in 1:09:52.  Best coloring so far was coloring 0 in step 2560 with 189 problems.  Time now 2017-07-06 17:16:00.
2905 steps done in 1:19:17.  Best coloring so far was coloring 0 in step 2905 with 188 problems.  Time now 2017-07-06 17:25:24.
3009 steps done in 1:22:07.  Best coloring so far was coloring 0 in step 3009 with 187 problems.  Time now 2017-07-06 17:28:14.
3012 steps done in 1:22:12.  Best coloring so far was coloring 0 in step 3012 with 186 problems.  Time now 2017-07-06 17:28:19.
3139 steps done in 1:25:40.  Best coloring so far was coloring 0 in step 3139 with 183 problems.  Time now 2017-07-06 17:31:47.
3173 steps done in 1:26:36.  Best coloring so far was coloring 0 in step 3173 with 181 problems.  Time now 2017-07-06 17:32:43.
3203 steps done in 1:27:25.  Best coloring so far was coloring 0 in step 3203 with 179 problems.  Time now 2017-07-06 17:33:32.
3255 steps done in 1:28:50.  Best coloring so far was coloring 0 in step 3255 with 178 problems.  Time n

,edge,color
0,"(0, 1)",1
1,"(0, 2)",1
2,"(0, 3)",1
3,"(0, 4)",0
4,"(0, 5)",0
5,"(0, 6)",0
6,"(0, 7)",1
7,"(0, 8)",0
8,"(0, 9)",1
9,"(0, 10)",0


# DO NOT TOUCH WHAT IS BELOW. 

In [ ]:
from setup import *
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import random

def main_random(Ramsey, num_vertices, num_steps, beta=2):
    start_time = datetime.datetime.now()
    print()

    def choose(n,k):
        return int(math.factorial(n)/math.factorial(k)/math.factorial(n-k))
    #The theory for this function comes from the Lesser Paper. Theorem 7 there provides very useful lower-bounds 
    #for the number of red edges found 
    def min_red_edges(Ramsey, num_vertices):
        sorted_Ramsey = np.sort(Ramsey)
        k = sorted_Ramsey[1]
        if len(sorted_Ramsey) != 2 or sorted_Ramsey[0] != 3:
            return(0)
        elif num_vertices <=2*k:
            return(num_vertices-k)
        elif num_vertices <= 5*k/2:
            return(3*num_vertices - 5*k)
        else:
            return(5*num_vertices - 10*k)
    min_red = min_red_edges(Ramsey, num_vertices)
    num_colors = len(Ramsey)
    Colors = np.arange(num_colors)
    Vertices = np.arange(num_vertices)
    Edges = list(it.combinations(Vertices,2))
    #reverse lookup for edges below.  Eg if slot 3 above contains edge (2,5), the the dict below has entry (2,5):3
    Edges_idx = dict((edge, idx) for idx,edge in enumerate(Edges)) 
    num_edges = len(Edges)

    threads_per_block = 1024
    vertices_per_clique = Ramsey
    edges_per_clique = np.array([choose(v,2) for v in vertices_per_clique])
    cliques_per_color = np.asarray([choose(num_vertices,v) for v in vertices_per_clique])
    blocks_per_color = np.ceil(cliques_per_color / threads_per_block).astype('uint32')
    num_blocks = blocks_per_color.sum()
    cliques_per_block = np.ceil(cliques_per_color / blocks_per_color).astype('uint32')
    #The objects below tells each block which color and how many cliques/edges it will monitor.
    #Note each vector is repetitive.  If color 0 gets 7 blocks, the first 7 entries will be the same
    block_color = np.repeat(Colors,blocks_per_color).astype('uint32')
    block_num_cliques = np.repeat(cliques_per_block,blocks_per_color).astype('uint32')
    block_edges_per_clique = np.repeat(edges_per_clique,blocks_per_color).astype('uint32')
    
    #The object below assigns each block to a list of cliques.  For simplicity while
    #we construct a single matrix with a lot of unused entries.
    #For example, edges_per_clique is different for blocks monitoring different colors.
    #We could make a more complex structure that handles this (see old stable version)
    #but that makes it harder to pass to the GPU.  Instead, we simply fill all "invalid"
    #unused entries with the placeholder "num_edges", which is 1 more the the largest
    #legal edge_idx (because Python indexes [0,1,2,...,num_edges-1]).
    #When we color later, we color these slots with the placeholder num_colors.
    assign_Blocks_to_Cliques = np.full([num_blocks,cliques_per_block.max(),edges_per_clique.max()],
                                       fill_value=num_edges, dtype='uint32')

    #Counters that that tracks the next open block and thread on each block
    next_open_block = 0    
    next_open_thread = np.zeros(num_blocks,dtype='int')
    for color, clique_size in enumerate(Ramsey):
        #Creates a generator to produce all cliques (the list of vertices).
        Cliques = it.combinations(Vertices,clique_size)

        #Makes the vector [0,1,2,...,num_blocks-1,0,1,2,...,num_blocks-1,....] of length num_cliques
        assign_Cliques_to_Blocks = np.arange(cliques_per_color[color]) % blocks_per_color[color]
        #randomizes assignment, but maintains clique counts
        np.random.shuffle(assign_Cliques_to_Blocks)
        #Starts at next open block
        assign_Cliques_to_Blocks += next_open_block
        
        for clique_Vertices, block in zip(Cliques,assign_Cliques_to_Blocks):
            #Gets the list of edges in this clique
            clique_Edges = list(it.combinations(clique_Vertices,2))
            #Converts it to edge_idx
            clique_Edges_idx = [Edges_idx[edge] for edge in clique_Edges]
            #Writes it to the correct block and next open thread on that block
            assign_Blocks_to_Cliques[block,next_open_thread[block],:edges_per_clique[color]] = clique_Edges_idx            
            next_open_thread[block] += 1
        next_open_block += blocks_per_color[color]

#     print("Ramsey");print(Ramsey);print("edges per clique");print(edges_per_clique);print("cliques per color");print(cliques_per_color);print("blocks per color");print(blocks_per_color);print("cliques per block");print(cliques_per_block)
#     for (idx, block)  in enumerate(assign_Blocks_to_Cliques):
#         print()
#         print("block = "+str(idx));
#         print("color = "+str(block_color[idx]));
#         print("num_cliques = "+str(block_num_cliques[idx]));
#         print("edges per clique = "+str(block_edges_per_clique[idx]));
#         print(block.shape, block.dtype);
#         display(block);
#         display(compare)
  
    
    #Code below sets up the GPU checker
    block_color_gpu = gpuarray.to_gpu(block_color)
    #block_num_cliques = gpuarray.to_gpu(block_num_cliques)
    block_edges_per_clique_gpu = gpuarray.to_gpu(block_edges_per_clique)
    assign_Blocks_to_Cliques_gpu = gpuarray.to_gpu(assign_Blocks_to_Cliques)    

    #mod = SourceModule("""
    #__global__ void find_problems(int *block_color, int *edges_per_clique, int *edges, int *coloring, int *Problems, int edges_per_thread)
    #{
    #   int color = block_color[blockIdx.x];
    #    int clique_idx = blockIdx.x*blockDim.x + threadIdx.x;
    #    int start = clique_idx*edges_per_thread;
    #    int end = start + edges_per_clique[blockIdx.x];
    #    int e = start;
    #    while((e < end) && (coloring[edges[e]] == color)){
    #        e++;
    #    }
    #    Problems[clique_idx] = (e >= end) ? 1 : 0;
    #}
    #""")
    kernel_code ="""
    #include <stdio.h>
    __global__ void find_problems(int *block_color, int *edges_per_clique, int *edges, int *coloring, int *Problems, int edges_per_thread, int cliques)
    {
        __shared__ int shared_coloring[shared_size];
        int color = block_color[blockIdx.x];
        int clique_idx = blockIdx.x*blockDim.x + threadIdx.x;
        if(threadIdx.x < shared_size)
        {
            shared_coloring[threadIdx.x] = coloring[threadIdx.x];
        }
        if(clique_idx < cliques)
        {
            //This weird way of starting and stopping allows us to traverse the 3D-array of assign_Blocks_to_Cliques easily
            int start = clique_idx*edges_per_thread;
            int end = start + edges_per_clique[blockIdx.x];
            int e = start;
            while((e < end) && (shared_coloring[edges[e]] == color))
            {
                e++;
            }
            //Problems[clique_idx] = 0;
            //(e >= end) ? atomicAdd(Problems[clique_idx],int 1) : atomicAdd(Problems[clique_idx],int 0);
            Problems[clique_idx] = (e >= end) ? 1 : 0;
        }
    }
    """
    
    regex_fixing_dictionary = {"shared_size":(num_edges+1)}

    for key, val in regex_fixing_dictionary.items():
        kernel_code = kernel_code.replace(str(key),str(val))
    mod = SourceModule(kernel_code)
    G, B, edges_per_thread = assign_Blocks_to_Cliques_gpu.shape
    edges_per_thread = np.uint32(edges_per_thread)
    grid_dimensions = (G,1)
    block_dimensions = (B,1,1)
    print("#blocks = gridDim.x = %d, cliques per block = threads per block = blockDim.x = %d, edges per thread = %d"%(G,B,edges_per_thread))
    func = mod.get_function("find_problems")
    def find_problems_cuda(coloring_gpu, printout=False, get_from_gpu=False):
        func(block_color_gpu, block_edges_per_clique_gpu, assign_Blocks_to_Cliques_gpu, coloring_gpu, Problems_gpu, edges_per_thread, np.uint32(cliques_per_color.sum()), block=block_dimensions, grid=grid_dimensions, shared=0)
        if printout == True:
            get_from_gpu = True
        if get_from_gpu == True:
            #print("getting from gpu")
            Problems_cpu = Problems_gpu.get()
            #cuda.memcpy_dtoh_async(Problems_gpu, Problems_cpu)
            if printout == True:
                print_problems(Problems_cpu)
        else:
            Problems_cpu = []
        return gpuarray.sum(Problems_gpu).get().astype('uint32'), Problems_cpu
    
    #The code below setups the serial version of this algorithm in pandas on the CPU.
    #It is much slower than the gpu version, but can be used in the absence of a GPU
    #and to verify that the algorithms give the same answers.
    #We msut create the "comparison" array.  This is a bit complicated.
    #We will discuss 2 arrays: compare and the coloring array.
    #First, recall that num_colors is one larger than the biggest legal color since
    #Python indexes [0,1,...,num_colors-1]
    #Now, fix a block and let c = block_color[block].
    #Consider the [block, clique, edge] entry of compare.  It equals:
    #c IF edge < num_edges_per_clique for that block
    #num_colors IF clique >= num_edges_per_clique for that block
    #Why?  In general there are extra rows and columns not associated to a valid edge.
    #When we color the graph later, they are filled with num_colors.
    #We do NOT want the "space fillers" to affect problem count.
    #In the extra rows, we see [c,c,...,c,num_colors,num_colors,...,num_colors] in compare
    #But in the coloring array, all entries will equal num_colors.
    #Thus, it is NOT counted as a problem because the first several slots disagree.
    #Thus, these extra rows can NEVER counts as problems cliques, as desired.
    #Now, consider the extra columns.  All entries will equal num_colors.  This is true
    #for BOTH compare AND the coloring array.  Thus, a row counts as a problem
    #if and only if the first num_edges_per_clique "valid" entries also match.
    #Thus the extra columns do NOT alter the "problem status" for valid rows, as desired.

    #compare = np.full_like(assign_Blocks_to_Cliques, fill_value=num_colors)
    #print(assign_Blocks_to_Cliques.shape)
    #for block in range(num_blocks):
    #    compare[block,:,:block_edges_per_clique[block]] = block_color[block]
    #
    #def find_problems_pandas(coloring, printout=False):
    #    X = coloring[assign_Blocks_to_Cliques]
    #    Y = (X == compare)
    #    Problems = np.all(Y,axis=-1)
    #    if printout == True:
    #        print_problems(Problems)
    #    return Problems.sum().astype('int'), Problems

    def print_problems(Problems):        
        Z = pd.DataFrame(Problems.astype('uint32'))
        Z.insert(0,'problems',Z.sum(axis=1))
        Z.insert(0,'color',block_color)
        Z = Z.T
        problem_idx = np.any(Z,axis=-1)
        problem_idx[:2] = True
        display(Z.ix[problem_idx,:])

    def print_status():
        now = datetime.datetime.now()
        elapsed = now - start_time
        print("%d steps done in %s.  Best coloring so far was step %d with %d problems.  Time now %s."
                   %(step,str(elapsed).split('.')[0],step_best,num_problems_best,str(now).split('.')[0]))
    def increase_red_edges():
        while(list(coloring_cpu).count(red)<min_red):
            idx = [i for i in range(len(coloring_cpu)) if coloring_cpu[i]!=red]
            coloring_cpu[random.choice(idx)] = red
    #Initialize the Markov chain
    coloring_cpu = np.random.choice(Colors, size=num_edges+1, replace=True).astype('uint32')
    coloring_cpu[num_edges] = num_colors
    #Recall this last slot holds is a placeholder to handlge "extra" slots.  See discussion
    #of serial pandas algorithm above.
    coloring_best = coloring_cpu.copy()
    coloring_gpu = gpuarray.to_gpu(coloring_cpu.copy())
    
    #Problems_current = np.zeros(assign_Blocks_to_Cliques.shape[:-1]).astype('uint32')    
    #Problems_gpu = gpuarray.GPUArray(assign_Blocks_to_Cliques.shape[:-1],dtype='uint32')
    Problems_gpu = gpuarray.to_gpu(np.zeros(assign_Blocks_to_Cliques.shape[:-1]).astype("uint32"))

    #num_problems_current, Problems_current = find_problems_pandas(coloring_cpu, printout=False)
    #num_problems_current, Problems_current = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
    num_problems_current, _ = find_problems_cuda(coloring_gpu, get_from_gpu=False, printout=False)    
    num_problems_proposed = num_problems_current    
    num_problems_best = num_problems_current
    #Problems_proposed = Problems_current.copy()
    #Problems_best = Problems_current.copy()
    red = min(vertices_per_clique)
    step = 0
    step_best = step
    
    loop_length = 100000
    loop_step = 0
    loops_done = 0
    start_compute = datetime.datetime.now()
    for i in range(num_steps):
        if num_problems_best == 0:
            break
        
        edge_idx = np.random.randint(0,num_edges)
        color_delta = np.random.randint(1,num_colors)
        edge_color_old = coloring_cpu[edge_idx]
        #edge_color_new = (edge_color_old + color_Deltas[i]) % num_colors
        edge_color_new = (edge_color_old + color_delta) % num_colors
        coloring_cpu[edge_idx] = edge_color_new
        if(list(coloring_cpu).count(red) < min_red):
            increase_red_edges()    
        coloring_gpu.set(coloring_cpu)

#         The code below check the pandas and cuda versions against each other.
#         It is commented out by default because it slows things down.
#         If you want to use it, you also need to uncomment several lines above to activate the pandas algorithm.

        #num_problems_proposed, Problems_proposed_pandas = find_problems_pandas(coloring_cpu)#, printout=True)
        #num_problems_proposed, Problems_proposed_cuda = find_problems_cuda(coloring_gpu, get_from_gpu=True, printout=False)
        #if np.all(Problems_proposed_pandas == Problems_proposed_cuda) == True:
        #    print("Pandas and Cuda agree!!")
        #else:
        #    raise Exception("Pandas and Cuda disagree :()")
            
        num_problems_proposed, _ = find_problems_cuda(coloring_gpu, get_from_gpu=False, printout=False)
        num_problems_diff = num_problems_current - num_problems_proposed
        if num_problems_diff >= 0:
             #print("Proposed is better.  Accepting.")            
            num_problems_current = num_problems_proposed
            #Problems_current = Problems_proposed.copy()
            if num_problems_proposed < num_problems_best:
                step_best = step
                coloring_best = coloring_cpu.copy()
                num_problems_best = num_problems_proposed
                #Problems_best = Problems_proposed.copy()
                print_status()
        else:            
            accept = np.exp(beta * num_problems_diff)            
            r = np.random.random()
            #print("Proposed is worse.  But I will accept it anyway if I draw a number less than %.3f.  I drew %.3f." % (accept,r))            
            if r <= accept:            
                #print("So I accept the move even though it is worse.")                
                num_problems_current = num_problems_proposed
                #Problems_current = Problems_proposed.copy()
            else:                
                #print("So I reject.")
                coloring_cpu[edge_idx] = edge_color_old
        step += 1
        loop_step += 1
        if(loop_step >= loop_length):
            loops_done += 1
            loop_step = 0
            print_status()
            compute_time = (datetime.datetime.now() - start_compute).seconds
            steps_done = loops_done*loop_length
            rate = steps_done / compute_time
            job_time = (num_steps-steps_done)/rate
            m, s = divmod(job_time,60)
            h, m = divmod(m,60)
            d, h = divmod(h,24)
            y, d = divmod(d,365)
            print("At %.0f colorings/second, it'll take me %d years %d days %d hours %d minutes and %d seconds to complete the remaining steps."%
                  (rate,y,d,h,m,s))

    

    print("FINISHED!!")
    coloring_cpu = coloring_best.copy()
    coloring_gpu.set(coloring_best)
    num_problems_best, _   = find_problems_cuda(coloring_gpu, get_from_gpu=False, printout=False)
    
    print()
    print_status()
    final_coloring = pd.DataFrame()
    final_coloring['edge'] = Edges
    final_coloring['color'] = coloring_best[:num_edges]
    display(final_coloring)
    return final_coloring

# YOU MAY NOW EDIT CODE AGAIN. DO NOT TOUCH WHAT IS DIRECTLY ABOVE ME

In [ ]:
Ramsey = [4,6]
num_vertices = 35

num_steps = 326#50000#This takes just under 18 hours to run
import datetime 
#Ramsey = [3,3,4]
#num_vertices = 30
#num_steps = 1000000000

beta = 1
bill = main_random(Ramsey, num_vertices, num_steps, beta=beta)